In [2]:
import requests
from pprint import pprint
import re
import lxml.html
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def log_error(line1, line2):
    _line1 = str(line1)
    _line2 = str(line2)
    diff = 0
    chars1 = len(_line1) + 10
    chars2 = len(_line2) + 10
    if chars1 > chars2:
        diff = chars1 - chars2
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line2 = '# ' + _line2
        else:
            _line2 = ' ' + _line2
        main_hashes = ''.join(['#' for i in range(chars1)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes}{_line2} {hashes}\n"                    
                    f"{main_hashes}\n")
        return(log_line)
   
    elif chars2 > chars1:
        diff = chars2 - chars1
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line1 = '# ' + _line1
        else:
            _line1 = ' ' + _line1
        main_hashes = ''.join(['#' for i in range(chars2)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    
                    f"{hashes}{_line1} {hashes}\n"
                    f"#### {_line2} ####\n"
                    f"{main_hashes}\n")
        return(log_line)
        
    else:
        hashes = '####'
        main_hashes = ''.join(['#' for i in range(chars2)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes} {_line2} {hashes}\n"
                    f"{main_hashes}\n")
        return(log_line)
        

In [4]:
# init session 
S = requests.Session()

# set URL
URL = "https://en.wikipedia.org/w/api.php"

In [ ]:
page_id = "64618"
topic = 'poet'

In [ ]:
# get sections
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "sections"
}

sectionsR = S.get(url=URL, params=PARAMS)
sectionDATA = sectionsR.json()



In [ ]:
sectionDATA

In [ ]:
sections = [sec['anchor'] for sec in sectionDATA["parse"]["sections"]]

In [ ]:
# create section list filtering out non-data sections
filter = [
    #'References',
    'See_also',
    'External_links'
]

sec_list = [sec for sec in sectionDATA["parse"]["sections"] if not sec['anchor'] in filter]


len(sec_list)

In [ ]:
# create nested section_dict dictionary from section list based on toclevel
# adding 'Z' section by hand consider refactor
section_dict = {}
is_new_parent = True
parent = ''
for i in range(len(sec_list) - 1):
    if sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 1:
        section_dict[sec_list[i]['anchor']] = []
    elif sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 2:
        section_dict[sec_list[i]['anchor']] = {}
        parent = sec_list[i]['anchor']
    elif sec_list[i]['toclevel'] == 2:
        section_dict[parent][sec_list[i]['anchor']] = []
section_dict['Z'] = []
#section_dict

In [ ]:
# make list of top level sections
secs = [sec for sec, val in section_dict.items()]


In [ ]:
# make section dict if no nested sections
section_dict = {sec['anchor']:[] for sec in sec_list}

In [ ]:
# get links
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "links"
}

linkR = S.get(url=URL, params=PARAMS)
linkDATA = linkR.json()

pprint(linkDATA["parse"]["links"])

In [ ]:
links = [link['*'] for link in linkDATA["parse"]["links"]]
for i, link in enumerate(links):
    print(i, link)

In [ ]:
# visualize link data
for link in links[3:4]:
    # get page info
    PARAMS = {
        "action": "query",
        "titles": link,
        "format": "json",
        "prop": "info",
        'inprop': "displaytitle"
    }

    infoR = S.get(url=URL, params=PARAMS)
    infoDATA = infoR.json()
    pprint(infoDATA)

In [ ]:
# get data
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "wikitext"
}

textR = S.get(url=URL, params=PARAMS)
textDATA = textR.json()


In [ ]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')

In [ ]:
for index, line in enumerate(lines):
    print(index, line)

In [ ]:
items = []
descriptions = []
with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines[77:291]):
        
        if re.match('\*\s*\[\[', line[0:5]):
            #print("%%%Match%%%")
            #print(line)
            try:
                line = re.sub(r'\<ref.*?\</ref>|\<ref.*\/>', '', line)
                
            except:
                topic_error_line = f"{topic} line sub parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                item = re.search(r'\[\[(.*?)\]\]', line).group(1)
                
            except:
                topic_error_line = f"{topic} item parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                description = re.search(r'\]\]\:*(.*)', line).group(1)
                
                
            except:
                topic_error_line = f"{topic} description parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            items.append(item)
            descriptions.append(description)
        else:
            continue
            #print("%%%No%%%")
            #print(line)

In [ ]:
items = [re.sub(r'\:wiktionary\:', '', item) for item in items]
items = [re.sub(r'\|.*', '', item) for item in items]
items

In [ ]:

item_infos = []
for index, title in enumerate(items):
    with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
        
        item_infos.append({
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        })


In [ ]:
# transfer item_infos to json file
import json
with open(topic + '_infos.json', 'w') as filepath:
    json.dump(item_infos, filepath)

In [ ]:
error_items = [item[topic] for item in item_infos if item['views'] == '#error#']

for index, title in enumerate(error_items):
    with open('info_error_log.txt', 'w+', encoding="utf-8") as filepath:
        #print(title)
        del_idx = 0
        for index, item in enumerate(item_infos):
            if item[topic] == title:
                del_idx = index
                #del item_infos[index]
        #print(item)
        
        
        if re.match(r'\W*', title):
            print('%%%')
            search = re.search(r'(\w+)?', title).group(1)
            print(search)
            title = search
        
        print(title)
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
  
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
            
        item_infos[del_idx] = {
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        }

In [ ]:
with open('mineral_infos.json', 'r') as jfile:
    data=jfile.read()

# parse file
item_infos = json.loads(data)

In [ ]:
_item_infos = item_infos
len(_item_infos)

In [ ]:
#item_infos = [{item['views']: int(item['views'].replace(',', ''))} for item in item_infos]
for index, item in enumerate(_item_infos):
    if item['views'] != '#error#':
        if item['watchers'] == 'Fewer than 30 watchers':
            item['watchers'] = '29'
        item['views'] = int(item['views'].replace(',', ''))
        item['watchers'] = int(item['watchers'].replace(',', ''))
        #item['edits'] = int(item['edits'].replace(',', ''))
        
        
        

In [ ]:
by_views = sorted(_item_infos, key=lambda k : k['views'], reverse=True)
for index, item in enumerate(by_views):
    item['view_rank'] = index

In [ ]:
by_edits = sorted(_item_infos, key=lambda k : k['edits'], reverse=True)
for index, item in enumerate(by_edits):
    item['edit_rank'] = index

In [ ]:
by_watchers = sorted(_item_infos, key=lambda k : k['watchers'], reverse=True)
for index, item in enumerate(by_watchers):
    item['watcher_rank'] = index

In [ ]:
for item in _item_infos:
    #item['total_rank'] = (item['view_rank'] + item['edit_rank'] + item['watcher_rank']) / 3
    item['total_rank'] = (item['view_rank'] + item['watcher_rank']) / 2

    

In [ ]:
by_total = sorted(_item_infos, key=lambda k : k['total_rank'], reverse=True)

In [ ]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['total_rank'], ascending=False).head(100)

In [ ]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['views'], ascending=False).head(100)

In [ ]:
min_df.sort_values(by=['watchers'], ascending=False).head(100)

In [ ]:
min_df.sort_values(by=['edits'], ascending=False).head(100)

In [ ]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')[3:-10] # [::-1]
this_sec = {}
sec = ''
sub_sec = ''
with open('error_lines.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines):
        if index == 23:
            print(line)
        if re.match('==\w', line[0:3]):
            sec = line.strip().replace("=", "")
            if len(this_sec) > 0:
                section_dict[secs[secs.index(sec) - 1]] = this_sec
            this_sec = {}
        if type(section_dict[sec]).__name__ == 'list':
            if re.match('\*\[\[', line[0:3]):
                try:
                    poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                    #print(poet)

                except:
                    print("##########################")
                    print("#### poet parse error ####")
                    print(poet_method, index, line)
                    filepath.write(line + '\n')
                    print(line[-2:])
                    print("##########################")

                date_m = re.search('\((.*)\)', line)

                try:
                    date = date_m.group(0)
                except:
                    print("##########################")
                    print("#### date parse error ####")
                    print(index, line)
                    filepath.write(line + '\n')
                    date = ""
                    print("##########################")
                try:
                    # description = re.search('\)*,*.*', line).group(1)
                    # description = re.search('\),+.*', line).group(1)
                    # description = re.search('\]\s.*\)(.*)', line).group(1)
                    description = re.search('\]\](\)*,*.*)', line).group(1)
                except Exception as e:
                    print("#################################")
                    print("#### description parse error ####")
                    print(e)
                    print(index, line)
                    filepath.write(line + '\n')
                    print("#################################")
                # print(this_sec)
                # print(sub_sec)
                section_dict[sec].append({
                    'poet': poet,
                    'date': date,
                    'description': description
                })
        else:
            #print(sec)
            if re.match('===\w', line[0:4]):
                sub_sec = line.strip().replace("=", "")
                #print(sub_sec)
            if line.strip().replace("=", "") in section_dict[sec]:
                #print(sub_sec)
                this_sec[sub_sec] = []
            if sub_sec != '':
                if re.match('\*\[\[', line[0:3]):
                    try:
                        poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                        #print(poet)
                    except:
                        print("##########################")
                        print("#### poet parse error ####")
                        print(index, line)
                        filepath.write(line + '\n')
                        print(line[-2:])
                        print("##########################")
                    date_m = re.search('\((.*)\)', line)
                    try:
                        date = date_m.group(0)
                    except Exception as e:
                        print("##########################")
                        print("#### date parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        date = ""
                        print("##########################")
                    try:
                        # description = re.search('\)*,*.*', line).group(1)
                        # description = re.search('\),+.*', line).group(0)
                        # description = re.search('\]\s.*\)(.*)', line).group(0)
                        description = re.search('\]\](\)*,*.*)', line).group(1)

                    except Exception as e:
                        print("#################################")
                        print("#### description parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        description = ""
                        print("#################################")

                    this_sec[sub_sec].append({
                        'poet': poet,
                        'date': date,
                        'description': description
                    })
